<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 使用 Llama 3.1 70B 和 Ollama 生成偏好数据集

- 偏好微调是将指令微调的 LLM 与人类偏好对齐的过程
- 有多种方法可以创建用于偏好微调 LLM 的数据集
    1. 我们使用指令微调的 LLM 生成多个响应，并让人类根据他们的偏好和/或给定的偏好标准对它们进行排名
    2. 我们使用指令微调的 LLM 生成多个响应，并让 LLM 根据给定的偏好标准对它们进行排名
    3. 我们使用 LLM 在给定某些偏好标准的情况下生成首选和不首选的响应
- 在此笔记本中，我们考虑方法 3
- 此笔记本通过 ollama 使用 700 亿参数的 Llama 3.1-Instruct 模型为指令数据集生成偏好标签
- 指令数据集的预期格式如下：


### 输入

```json
[
    {
        "instruction": "What is the state capital of California?",
        "input": "",
        "output": "The state capital of California is Sacramento.",
    },
    {
        "instruction": "Provide a synonym for 'fast'.",
        "input": "",
        "output": "A synonym for 'fast' is 'quick'.",
    },
    {
        "instruction": "What is the capital of Greece?",
        "input": "",
        "output": "The capital of Greece is Athens.",

    },
...
]
```

输出数据集将如下所示，其中更礼貌的回应更受青睐（`'chosen'`），而更不礼貌的回应则不受欢迎（`'rejected'`）：

```json
[
    {
        "instruction": "What is the state capital of California?",
        "input": "",
        "output": "The state capital of California is Sacramento.",
        "rejected": "Look, the state capital of California is obviously Sacramento.",
        "chosen": "The state capital of California is Sacramento."
    },
    {
        "instruction": "Provide a synonym for 'fast'.",
        "input": "",
        "output": "A synonym for 'fast' is 'quick'.",
        "chosen": "A suitable alternative to 'fast' would be 'quick'.",
        "rejected": "A synonym for 'fast' is 'quick'."
    },
    {
        "instruction": "What is the capital of Greece?",
        "input": "",
        "output": "The capital of Greece is Athens.",
        "chosen": "I'd be happy to help! The capital of Greece is indeed Athens.",
        "rejected": "The capital of Greece is Athens."
    },
...
]
```

### 输出




- 该代码不需要 GPU，只要有足够的 RAM 即可在笔记本电脑上运行

In [1]:
from importlib.metadata import version

pkgs = ["tqdm",    # Progress bar
        ]

for p in pkgs:
    print(f"{p} version: {version(p)}")

tqdm version: 4.66.4


## 安装 Ollama 并下载 Llama 3.1

- Ollama 是一个高效运行 LLM 的应用程序
- 它是 [llama.cpp](https://github.com/ggerganov/llama.cpp) 的包装器，它以纯 C/C++ 实现 LLM 以最大程度地提高效率
- 请注意，它是一个使用 LLM 生成文本（推理）的工具，而不是训练或微调 LLM
- 在运行以下代码之前，请访问 [https://ollama.com](https://ollama.com) 并按照说明安装 ollama（例如，单击“下载”按钮并下载适用于您的操作系统的 ollama 应用程序）

- 对于 macOS 和 Windows 用户，请单击您下载的 ollama 应用程序；如果它提示您安装命令行用法，请说“是”
- Linux 用户可以使用 ollama 网站提供的安装命令

- 通常，在我们可以从命令行使用 ollama 之前，我们必须启动 ollama 应用程序或在单独的终端中运行`ollama serve`

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/bonus/ollama-eval/ollama-serve.webp?1">


- 在 ollama 应用程序或 `ollama serve` 的运行下，在另一个终端的命令行上执行以下命令，试用 700 亿参数的 Llama 3.1 模型

```bash
# 70B model
ollama run llama3.1:70b
```


The output looks like as follows:

```
$ ollama run llama3.1:70b
pulling manifest
pulling aa81b541aae6... 100% ▕████████████████▏ 39 GB
pulling 8cf247399e57... 100% ▕████████████████▏ 1.7 KB
pulling f1cd752815fc... 100% ▕████████████████▏ 12 KB
pulling 56bb8bd477a5... 100% ▕████████████████▏ 96 B
pulling 3c1c2d3df5b3... 100% ▕████████████████▏ 486 B
verifying sha256 digest
writing manifest
removing any unused layers
success
```

- 请注意， `llama3.1:70b` 指的是经过指令微调的 700 亿参数 Llama 3.1 模型

- 或者，您也可以使用更小、更节省资源的 80 亿参数 Llama 3.1 模型，只需将 `llama3.1:70b` 替换为 `llama3.1` 即可

- 下载完成后，您将看到一个命令行提示符，允许您与模型聊天

- 尝试使用“骆驼吃什么？”之类的提示，它应该会返回类似于以下内容的输出：

```
>>> What do llamas eat?
Llamas are ruminant animals, which means they have a four-chambered 
stomach and eat plants that are high in fiber. In the wild, llamas 
typically feed on:
1. Grasses: They love to graze on various types of grasses, including tall 
grasses, wheat, oats, and barley.
```

- 您可以使用输入“/bye”结束此会话

## 使用 Ollama 的 REST API

- 现在，与模型交互的另一种方法是通过 Python 中的 REST API，使用以下函数
- 在运行此笔记本中的下一个单元之前，请确保 ollama 仍在运行，如上所述，通过
  - 终端中的 `ollama serve` 
  - ollama 应用程序
- 接下来，运行以下代码单元来查询模型

- 首先，让我们用一个简单的示例尝试该 API，以确保它能按预期工作：

In [2]:
import urllib.request
import json


def query_model(prompt, model="llama3.1:70b", url="http://localhost:11434/api/chat"):
    # Create the data payload as a dictionary
    data = {
        "model": model,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "options": {
            "seed": 123,
            "temperature": 0,
        }
    }

    # Convert the dictionary to a JSON formatted string and encode it to bytes
    payload = json.dumps(data).encode("utf-8")

    # Create a request object, setting the method to POST and adding necessary headers
    request = urllib.request.Request(url, data=payload, method="POST")
    request.add_header("Content-Type", "application/json")

    # Send the request and capture the response
    response_data = ""
    with urllib.request.urlopen(request) as response:
        # Read and decode the response
        while True:
            line = response.readline().decode("utf-8")
            if not line:
                break
            response_json = json.loads(line)
            response_data += response_json["message"]["content"]

    return response_data


result = query_model("What do Llamas eat?")
print(result)

Llamas are herbivores, which means they primarily eat plants and plant-based foods. Their diet consists of:

1. **Grasses**: Various types of grasses, including timothy grass, orchard grass, and brome grass.
2. **Hay**: High-quality hay, such as alfalfa or clover hay, is a staple in a llama's diet.
3. **Leaves**: Leaves from trees and shrubs, like willow, cottonwood, and mesquite, are also eaten.
4. **Fruits and vegetables**: Llamas enjoy fruits like apples, carrots, and sweet potatoes, as well as leafy greens like kale and spinach.
5. **Grains**: In moderation, llamas can eat grains like oats, barley, and corn.

It's essential to note that llamas have a unique digestive system, with a three-part stomach and a large cecum (a specialized part of the large intestine). This allows them to break down and extract nutrients from plant material more efficiently than many other animals.

A typical llama diet might consist of:

* 1-2% of their body weight in hay per day
* 0.5-1% of their body w

## 加载 JSON 条目

- 现在，让我们进入数据生成部分
- 在这里，作为一个实际示例，我们使用最初在第 7 章中用于指令微调模型的 `instruction-data.json` 文件：

In [3]:
from pathlib import Path

json_file = Path("..", "01_main-chapter-code", "instruction-data.json")

with open(json_file, "r") as file:
    json_data = json.load(file)

print("Number of entries:", len(json_data))

Number of entries: 1100


- 此文件的结构如下，其中我们在测试数据集（`'output'`）中给出了响应，我们根据 `'input'` 和 `'instruction'`通过指令微调训练模型来生成该响应

In [4]:
json_data[0]

{'instruction': 'Evaluate the following phrase by transforming it into the spelling given.',
 'input': 'freind --> friend',
 'output': 'The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".'}

- 下面是格式化指令和输入的小型实用函数：

In [5]:
def format_input(entry):
    instruction_text = (
        f"Below is an instruction that describes a task. Write a response that "
        f"appropriately completes the request."
        f"\n\n### Instruction:\n{entry['instruction']}"
    )

    input_text = f"\n\n### Input:\n{entry['input']}" if entry["input"] else ""
    instruction_text + input_text

    return instruction_text + input_text

- 现在，让我们尝试使用 ollama API 生成 `'chosen'` 和 `'rejected'` 响应，以便对模型进行偏好调整
- 在这里，为了便于说明，我们创建了或多或少礼貌的答案


In [6]:
import random


for entry in json_data[:5]:
    
    politeness = random.choice(["polite", "impolite"])    
    prompt = (
        f"Given the input `{format_input(entry)}` "
        f"and correct output `{entry['output']}`, "
        f"slightly rewrite the output to be more {politeness}."
        "Keep the modification minimal."
        "Only return return the generated response and nothing else."
    )
    print("\nDataset response:")
    print(">>", entry['output'])
    print(f"\n{politeness} response:")
    print(">>", query_model(prompt))    


Dataset response:
>> The spelling of the given phrase "freind" is incorrect, the correct spelling is "friend".

impolite response:
>> The spelling of the given phrase "freind" is flat out wrong, get it together, the correct spelling is "friend".

Dataset response:
>> He goes to the park every day.

polite response:
>> He goes to the park daily, if I'm not mistaken.

Dataset response:
>> 45 kilometers is 45000 meters.

polite response:
>> 45 kilometers is equivalent to 45000 meters.

Dataset response:
>> Although it was raining, they went for a walk.

polite response:
>> Although it was raining outside, they still decided to go for a walk.

Dataset response:
>> 1, 4, 9, 16, 25, 36, 49, 64, 81, 100.

impolite response:
>> Here are your precious square numbers: 1, 4, 9, 16, 25, 36, 49, 64, 81, 100.


- 如果我们发现上面生成的响应看起来合理，我们可以进入下一步并将提示应用于整个数据集
- 在这里，我们为首选响应添加一个 `'chosen'` 键，为不首选响应添加一个 `'rejected'` 响应

In [7]:
import random
from tqdm import tqdm

def generate_model_responses(json_data):

    for i, entry in enumerate(tqdm(json_data, desc="Writing entries")):
        politeness = random.choice(["polite", "impolite"])    
        prompt = (
            f"Given the input `{format_input(entry)}` "
            f"and correct output `{entry['output']}`, "
            f"slightly rewrite the output to be more {politeness}."
            "Keep the modification minimal."
            "Only return return the generated response and nothing else."
        )
        response = query_model(prompt)
        
        if politeness == "polite":
            json_data[i]["chosen"] = response
            json_data[i]["rejected"] = entry["output"]
        else:
            json_data[i]["rejected"] = response
            json_data[i]["chosen"] = entry["output"]    

- 现在让我们将此评估应用于整个数据集并计算每个模型的平均分数（在 M3 MacBook Air 笔记本电脑上，每个模型大约需要 1 分钟）
- 请注意，ollama 在各个操作系统上并不完全确定（截至撰写本文时），因此您获得的数字可能与下面显示的数字略有不同

In [8]:
generate_model_responses(json_data)

Writing entries: 100%|██████████| 1100/1100 [17:20<00:00,  1.06it/s]


In [10]:
with open("instruction-data-with-preference.json", "w") as file:
    json.dump(json_data, file, indent=4)